In [1]:
%run data.ipynb    # load data
%run gauss.ipynb

In [2]:
import math
import sympy as sy
from scipy.optimize import fmin
import numpy as np

In [3]:
def gibbs_energy_i(i_name, T):
    x_t = T / 10000
    data = bank_td[i_name]
    phi = data['f1'] + data['f2'] * math.log(x_t) + data['f3'] / (x_t ** 2) + data['f4'] / x_t \
        + data['f5'] * x_t + data['f6'] * x_t ** 2 + data['f7'] * x_t ** 3
    gibbs = data['H'] - phi * T
    return gibbs

def G0(name, T):
    return gibbs_energy_i(name, T)

def G(name, T, PG, P, R=8314, delta=0.01):
    return D(name, T) * (PG[name] - P[name]) / R * T * delta

Функция gibbs_energy_i - энергия Гиббса i-ой компоненты (из формулы 27)

Функция G используется для нахождения межфазых мольных потоков активных компонент на поверхности вещества (из формулы 1)

In [4]:
def eps(name):
    return math.sqrt(bank_td[name]['epsil'] * bank_td['N2']['epsil'])

def mu(name):
    return 2 * bank_td[name]['mu'] * bank_td['N2']['mu'] / (bank_td[name]['mu'] + bank_td['N2']['mu'])

def omega(name, T):
    return 1.074 * ((T / eps(name)) ** (-0.1604))

def sigma(name):
    return (bank_td[name]['sigma'] + bank_td['N2']['sigma']) / 2

def D(name, T):
    bott = Pa * sigma(name) * omega(name, T) * math.sqrt(mu(name))
    return 0.02628 * (T ** 1.5) / bott

Функция D используется для расчета коэффициентов диффузии газообразных компонент (из формулы 29)

In [5]:
def new_zip(x, y):
    return list(zip(x, y))

def equalVector(a, b, eps):
    return sum([(a[i] - b[i])**2 for i in range(len(a))]) < eps

def mul_and_add(a, b, c):
    return [a[i] + b[i] * c for i in range(len(a))]

def minimumF(Funcs, Vars, xk, pk):
    sumFuncs = sy.sympify(0)
    for func in Funcs:
        sumFuncs += func ** 2
    sumFuncs = sy.lambdify(Vars, sumFuncs)
    xk = np.array(xk)
    pk = np.array(pk)
    return lambda a : sumFuncs(*(xk + pk * a[0]))

def newton_method(funcs, var, x0, eps=1e-20, ITER=150):
    j = funcs.jacobian(var)
    xk1 = sy.zeros(len(x0), 1)
    xk = x0
    iterations = 0
    while not equalVector(xk, xk1, eps) and iterations < ITER:
        xx = new_zip(var, xk)
        jk = j.subs(xx)
        fk = funcs.subs(xx)
        dk = gaussMethod(jk.tolist(), (fk.T * -1.0).tolist()[0])
        alpha = fmin(minimumF(funcs, var, xk, dk), [0], disp=False)[0]
        xk1 = mul_and_add(xk, dk, alpha)
        
        xk, xk1 = xk1, xk
        iterations += 1
    print(iterations)
    return xk

Функция newton_method используется для решения системы нелинейных уравнений (используется "универсальный метод". 

Принимаемые параметры: funcs - правые части системы, var - переменные из системы

Для нахождения вектора $\Delta x^{k} = x^{k + 1} - x^{k}$ решается СЛАУ $J_{k}*\Delta x^{k} = F(x^k)$ при помощи метода Гаусса.

Затем находится локальный минимум функции $F(x^k + \alpha \Delta x^k)$, где $F(x_1, x_2, ..., x_n) = \sum_{i=1}^{n}(f_i(x_1, x_2, ..., x_n))^2$

Затем вычисляется $x_{k+1} = x_k + \Delta x^{k}$

Критерий остановки метода - по близости 2х последовательных приближений. Максимальное количество итераций = 150